In [11]:
import findspark
findspark.init('/opt/spark')

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc

spark = SparkSession.builder \
    .appName("WDI Data Exploration") \
    .getOrCreate()

# COUNTRIES

In [12]:
file_path = "data/Countries.csv"
df_countries = spark.read.csv(file_path, header=True, inferSchema=True)

print("Rows:", df_countries.count())
print("Columns:", len(df_countries.columns))
print("Schema:")
df_countries.printSchema()

Rows: 265
Columns: 31
Schema:
root
 |-- Country Code: string (nullable = true)
 |-- Short Name: string (nullable = true)
 |-- Table Name: string (nullable = true)
 |-- Long Name: string (nullable = true)
 |-- 2-alpha code: string (nullable = true)
 |-- Currency Unit: string (nullable = true)
 |-- Special Notes: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Income Group: string (nullable = true)
 |-- WB-2 code: string (nullable = true)
 |-- National accounts base year: string (nullable = true)
 |-- National accounts reference year: integer (nullable = true)
 |-- SNA price valuation: string (nullable = true)
 |-- Lending category: string (nullable = true)
 |-- Other groups: string (nullable = true)
 |-- System of National Accounts: string (nullable = true)
 |-- Alternative conversion factor: string (nullable = true)
 |-- PPP survey year: string (nullable = true)
 |-- Balance of Payments Manual in use: string (nullable = true)
 |-- External debt Reporting status: str

In [13]:
df_countries.show()

+------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+-------------------+---------+---------------------------+--------------------------------+--------------------+----------------+------------+---------------------------+-----------------------------+---------------+---------------------------------+------------------------------+--------------------+-----------------------------+-------------------------------+------------------------+-----------------------+-------------------------------------------------+---------------------------+--------------------------+----------------------+-----------------+----------------------------+
|Country Code|          Short Name|          Table Name|           Long Name|2-alpha code|       Currency Unit|       Special Notes|              Region|       Income Group|WB-2 code|National accounts base year|National accounts reference year| SNA price val

# INDICATORS

In [14]:
file_path = "data/Indicators.csv"
df_indicators = spark.read.csv(file_path, header=True, inferSchema=True)

print("Rows:", df_indicators.count())
print("Columns:", len(df_indicators.columns))
print("Schema:")
df_indicators.printSchema()

Rows: 1509
Columns: 20
Schema:
root
 |-- Series Code: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- Indicator Name: string (nullable = true)
 |-- Short definition: string (nullable = true)
 |-- Long definition: string (nullable = true)
 |-- Unit of measure: string (nullable = true)
 |-- Periodicity: string (nullable = true)
 |-- Base Period: string (nullable = true)
 |-- Other notes: string (nullable = true)
 |-- Aggregation method: string (nullable = true)
 |-- Limitations and exceptions: string (nullable = true)
 |-- Notes from original source: string (nullable = true)
 |-- General comments: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Statistical concept and methodology: string (nullable = true)
 |-- Development relevance: string (nullable = true)
 |-- Related source links: string (nullable = true)
 |-- Other web links: string (nullable = true)
 |-- Related indicators: string (nullable = true)
 |-- License Type: string (nullable = true)



In [15]:
df_indicators.show(5)

+-----------------+--------------------+--------------------+----------------+--------------------+---------------+-----------+-----------+--------------------+------------------+--------------------------+--------------------------+----------------+--------------------+-----------------------------------+---------------------+--------------------+---------------+--------------------+--------------------+
|      Series Code|               Topic|      Indicator Name|Short definition|     Long definition|Unit of measure|Periodicity|Base Period|         Other notes|Aggregation method|Limitations and exceptions|Notes from original source|General comments|              Source|Statistical concept and methodology|Development relevance|Related source links|Other web links|  Related indicators|        License Type|
+-----------------+--------------------+--------------------+----------------+--------------------+---------------+-----------+-----------+--------------------+------------------+---

# WDI DATA

In [16]:
file_path = "data/Data.csv"
df_data = spark.read.csv(file_path, header=True, inferSchema=True)

print("Rows:", df_data.count())
print("Columns:", len(df_data.columns))
print("Schema:")
df_data.printSchema()

Rows: 401394
Columns: 69
Schema:
root
 |-- Country Name: string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- Indicator Name: string (nullable = true)
 |-- Indicator Code: string (nullable = true)
 |-- 1960: double (nullable = true)
 |-- 1961: double (nullable = true)
 |-- 1962: double (nullable = true)
 |-- 1963: double (nullable = true)
 |-- 1964: double (nullable = true)
 |-- 1965: double (nullable = true)
 |-- 1966: double (nullable = true)
 |-- 1967: double (nullable = true)
 |-- 1968: double (nullable = true)
 |-- 1969: double (nullable = true)
 |-- 1970: double (nullable = true)
 |-- 1971: double (nullable = true)
 |-- 1972: double (nullable = true)
 |-- 1973: double (nullable = true)
 |-- 1974: double (nullable = true)
 |-- 1975: double (nullable = true)
 |-- 1976: double (nullable = true)
 |-- 1977: double (nullable = true)
 |-- 1978: double (nullable = true)
 |-- 1979: double (nullable = true)
 |-- 1980: double (nullable = true)
 |-- 1981: double (nullabl

In [17]:
df_data.show(5)

+--------------------+------------+--------------------+-----------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+----+
|        Country Name|Country Code|      Indicator Name|   Indicator Code|1960|1961|1962|1963|1964|1965|1966|1967|1968|1969|1970|1971|1972|1973|1974|1975|1976|1977|1978|1979|1980|1981|1982|1983|1984|1985|1986|1987|1988|1989|1990|1991|1992|1993|1994|1995|1996|1997|1

In [18]:
years = [str(y) for y in range(1960, 2024)]
countries      = ["DEU", "FRA", "ITA", "GBR", "ESP"]
indicators = {
    "economic": ["NY.GDP.MKTP.CD","NY.GDP.PCAP.CD","NY.GDP.MKTP.KD.ZG",
                 "NY.GNP.PCAP.CD","SI.POV.GINI","FP.CPI.TOTL.ZG"],
    "education": ["SE.XPD.TOTL.GB.ZS","SE.XPD.TOTL.GD.ZS","SE.SEC.ENRR","SE.TER.ENRR",
                  "SE.TER.CUAT.BA.ZS","SE.TER.CUAT.MS.ZS","SE.TER.CUAT.DO.ZS",
                  "SE.TER.ENRL.TC.ZS","SE.XPD.CPRM.ZS","SE.XPD.CSEC.ZS","SE.XPD.CTER.ZS",
                  "SE.XPD.PRIM.PC.ZS","SE.XPD.SECO.PC.ZS","SE.XPD.TERT.PC.ZS"],
    "energy": ["EG.USE.PCAP.KG.OE","EG.USE.ELEC.KH.PC","EG.ELC.NUCL.ZS",
               "EG.ELC.RNEW.ZS","EG.GDP.PUSE.KO.PP.KD"],
    "research_dev": ["GB.XPD.RSDV.GD.ZS","SP.POP.SCIE.RD.P6","SP.POP.TECH.RD.P6",
                     "IP.PAT.RESD","IP.PAT.NRES","IP.JRN.ARTC.SC","IT.NET.USER.ZS",
                     "IT.CEL.SETS.P2","IT.NET.BBND.P2","IT.NET.SECR.P6",
                     "TX.VAL.TECH.MF.ZS","TX.VAL.TECH.CD"],
    "labour": ["SL.UEM.TOTL.ZS","SL.UEM.1524.ZS"],
    "health": ["SP.DYN.LE00.IN","SH.XPD.CHEX.GD.ZS","SP.POP.65UP.TO.ZS"]
}

all_indicators = [code for group in indicators.values() for code in group]

df_focus = (df_data
            .filter(col("Country Code").isin(countries))
            .filter(col("Indicator Code").isin(all_indicators)))

print("Rows after filter:", df_focus.count())

Rows after filter: 210


In [19]:
df_focus.show(10)

+------------+------------+--------------------+-----------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+----+
|Country Name|Country Code|      Indicator Name|   Indicator Code|1960|1961|1962|1963|1964|1965|1966|1967|1968|1969|1970|1971|1972|1973|1974|1975|1976

In [20]:
# 1. Ranking of countries by a specific indicator in a given year
target_indicator = "EG.USE.PCAP.KG.OE"
target_year = "2022"

df_focus \
    .filter(col("Indicator Code") == target_indicator) \
    .select(col("Country Code"), col(target_year)) \
    .orderBy(desc(target_year)) \
    .show(5, False)

+------------+------------------+
|Country Code|2022              |
+------------+------------------+
|DEU         |3236.728451774681 |
|FRA         |3102.4031092675436|
|ITA         |2411.651724173114 |
|ESP         |2407.9957275008187|
|GBR         |2257.1375878230147|
+------------+------------------+



In [21]:
# 2. Show full wide table for health indicators
health_codes = ["SP.DYN.LE00.IN", "SH.XPD.CHEX.GD.ZS", "SP.POP.65UP.TO.ZS"]

df_focus \
    .filter(col("Indicator Code").isin(health_codes)) \
    .select("Country Name", "Country Code", "Indicator Name", "Indicator Code", *years) \
    .show(truncate=False)

+--------------+------------+----------------------------------------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+

In [22]:
# 3. Show all 'energy' indicators for a given country in wide form
category = "energy"
country = "ESP"

energy_codes = indicators[category]
years    = [str(y) for y in range(1964, 2025)]

df_focus \
    .filter((col("Country Code") == country) & col("Indicator Code").isin(energy_codes)) \
    .select("Country Name", "Country Code", "Indicator Name", "Indicator Code", *years) \
    .show(truncate=False)


+------------+------------+-------------------------------------------------------------------------+--------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----+
|Country Name|Country Code|Indicator Name                                                       